In [1]:
import pandas as pd

import requests
import whois
import virustotal_python
from requests_html import HTMLSession
from collections import Counter
from urllib.parse import urlparse
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from urllib3.exceptions import InsecureRequestWarning
from urllib3 import disable_warnings

disable_warnings(InsecureRequestWarning)


options = Options()
options.headless = True
 
from collections import defaultdict
from urllib.request import urlopen, URLError
import socket


In [2]:
domain = 'reddit.com'

def get_pagerank(domain):
    headers = {'API-OPR':'4ok88wgckg8o0cgcswo4gc4kkc0sgocw0woww4o0'}
    url = 'https://openpagerank.com/api/v1.0/getPageRank?domains%5B0%5D=' + domain
    request = requests.get(url, headers=headers)
    if request.status_code == 200:
        result = request.json()
        if result['status_code'] == 200:
            pr_dec = result['response'][0]['page_rank_decimal']
            rank = result['response'][0]['rank']

            return pr_dec, rank

def whois_checker(domain):
    w = whois.whois(domain)
    domain_registrar = w.get('registrar')
    postal_code = w.get('registrant_postal_code')
    country = w.get('country')

    return  domain_registrar, postal_code, country

def virus_scan(domain):
    with virustotal_python.Virustotal("242305dedab935c6a39736d0749ae88f2b68f1e2f0c8e82c81d8955c5fd194d3") as vtotal:
        resp = vtotal.request(f"domains/{domain}")
        virus_data = resp.json()['data']['attributes']['last_analysis_stats']
        return virus_data['malicious']


def link_checker(domain):
    session = HTMLSession()
    
    r = session.get("https://"+domain, verify=False)
    unique_netlocs = Counter(urlparse(link).netloc for link in r.html.absolute_links)
    outbound_n = 0
    local_n = 0

    for link in unique_netlocs:
        # print(link, unique_netlocs[link])
        if domain.lower() in link:
            local_n += unique_netlocs[link]
        else:
            outbound_n += unique_netlocs[link]

    return local_n, local_n + outbound_n



def check_login(domain, driver):
    wp_xpath = "//a[starts-with(@href, 'https://wordpress.org')]"
    try:
        displayed = driver.find_element('xpath', wp_xpath).is_displayed()

        return displayed
    except:

        return False

def check_page_source(domain, driver):
    try:
        driver.get(f"https://{domain}/wp-admin")
        wp_pg = "wp-content" in driver.page_source
        return wp_pg
    except:

        return False

def check_wp(domain):

    service_obj = Service(executable_path="C:/Users/cvaal/chromedriver.exe")
    driver = webdriver.Chrome(options=options, service=service_obj)
    driver.implicitly_wait(1)
    driver.get(f"https://{domain}/wp-admin")
    login = check_login(domain, driver)
    pg_source = check_page_source(domain, driver)
    wp = None
    if login == False and pg_source == False:
        wp = False
    else:
        wp = True

    driver.close()
    return wp


def check_url( url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'}
    response = requests.get('https://'+url, timeout=5, headers=headers)
   
    return response.status_code


def get_site_data(domain):
    # try:
    # print(check_url(domain))
    # if check_url(domain) == True:
    # try:
    pr_dec, rank = get_pagerank(domain)
    domain_registrar, postal_code, country = whois_checker(domain)
    malicious = virus_scan(domain)
    local_n, total_links= link_checker(domain)
    wp_check = check_wp(domain)
    return domain, pr_dec, rank, domain_registrar, postal_code, country, malicious, local_n, total_links, wp_check
# except:
    #     return('site down')
    # else:
    #     return 'Site down'
    # # except:
    # #     return 'Site down'
 

In [3]:
with open('FakeNews_URLs.txt', 'r') as f:
    fakenews_sites = f.read().split('\n')
    fakenews_sites = [site.strip() for site in fakenews_sites]


def get_and_organize_data(domain):
    columns = ['Domain', 'Page rank decimal', 'Site Rank', 'Domain registrar', 'Postal code', 'Country of origin', 'Malicious', 'No. of Local links','Total links','Wordpress?']
    site_df_dict = defaultdict(list)
    site_data = get_site_data(domain.lower())
    print(site_data)
    if type(site_data) == tuple:
        for column, data in zip(columns, site_data):
            site_df_dict[column].append(data)
         
        return pd.DataFrame(site_df_dict)
    else:
        print('site down')



In [4]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
import joblib


ct = joblib.load('SitePreprocessor.joblib')
clf = joblib.load('XGBoostClassifier.joblib')

def predict(domain):
    if 'http' in domain:
        domain = urlparse(domain).netloc.strip("www.")
    
    print('scanning:' + domain)
    data = get_and_organize_data(domain) #dataframe obj
    domains = data['Domain'].values
    features_df = data.drop('Domain', axis=1)
    

    result = clf.predict(features_df)
    if result > 0.5:
        print(f'{domain} : reliable site')
    else:
        print(f'{domain} : Unreliable site')

#idea: create instance of organizing object everytime the system is ran, collects all the links and returns a full dataframe

In [5]:
with open('FilipinoRealNews.txt', 'r') as f:
    fil_real_sites = f.read().split('\n')
    fil_real_sites= [site.strip() for site in fil_real_sites]

In [6]:
for site in fil_real_sites:
    predict(site)

scanning:businessmirror.com.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('businessmirror.com.ph', 4.64, '50443', None, None, None, 0, 479, 493, True)
businessmirror.com.ph : Unreliable site
scanning:bworldonline.com
('bworldonline.com', 5.06, '18162', 'Network Solutions, LLC', '1112', 'PH', 0, 146, 158, True)
bworldonline.com : reliable site
scanning:tribune.net.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('tribune.net.ph', 4.61, '55796', None, None, None, 0, 152, 160, True)
tribune.net.ph : Unreliable site
scanning:malaya.com.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('malaya.com.ph', 4.28, '274928', None, None, None, 0, 264, 269, True)
malaya.com.ph : Unreliable site
scanning:mb.com.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('mb.com.ph', 4.8, '30852', None, None, None, 0, 169, 194, True)
mb.com.ph : reliabl

In [8]:
predict('https://www.officialgazette.gov.ph/')

scanning:officialgazette.gov.ph
Error trying to connect to socket: closing socket - [Errno 11001] getaddrinfo failed
('officialgazette.gov.ph', 4.58, '62421', None, None, None, 0, 59, 72, False)
officialgazette.gov.ph : Unreliable site


In [ ]:
site_df_dict = defaultdict(list)

for rating, domains in site_dict.items():
    for i, domain in enumerate(domains):
        print(f'Analyzing Site {i+1}: {domain} of rating: {rating}')
        site_data = get_site_data(domain.lower(), rating)
        if type(site_data) == tuple:
            for column, data in zip(columns, site_data):
                if type(data) != list:
                    site_df_dict[column].append(data)
                else:
                    if any(site in data for site in fakenews_sites):
                        site_df_dict[column].append(True)
                    else:
                        site_df_dict[column].append(False)
            print('Data Gathered')
        else:
            print('site down')



NameError: name 'defaultdict' is not defined